<a href="https://colab.research.google.com/github/artms-18/tensorflow_fundamentals/blob/main/Intro_to_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundamentals in Tensorflow

NLP has the goal of deriving information out of natural language (could be sequences text or speech).

Another common term for NLP problems is sequence to sequence problems (seq2seq)

## Check for GPU

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-d1a39683-35f1-9f05-30d6-3746590b5730)


## Get helper functions

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import a series of helper function for the notebook

from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2021-05-26 19:42:50--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-05-26 19:42:51 (94.3 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataaset

The dataset we're going to be using is Kaggle's introduction to NLP dataset

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

unzip_data("nlp_getting_started.zip")

--2021-05-26 19:42:53--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.197.128, 74.125.20.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.197.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2021-05-26 19:42:53 (103 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

To visualie our text samples, we first have to read them in, one way to do so would be to use Python: 

Another way to do this is to use pandas.

(If text dataset is very large, cannot ise pandas (will tkaes of to much RAM).



In [4]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv('test.csv')

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_df['text'][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [6]:
# Shuffle training dataframe

train_df_shuffled = train_df.sample(frac = 1, random_state = 42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
# What does the test dataframe look like?

test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [9]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [10]:
# Let's visualize some random training examples

import random

random_index = random.randint(0, len(train_df)-5) # create random indexes not higher than the total number of samples

for row in train_df_shuffled[['text', 'target']][random_index: random_index + 5].itertuples():
  _, text, target = row #gets rid of index
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 0 (not real disaster
Text:
@RaynbowAffair Editor In Chief @DiamondKesawn Releases Issue #7 http://t.co/7mzYcU2IHo of #RAmag. #Fashion #Models and #Mayhem

---

Target: 0 (not real disaster
Text:
Do you have an emergency drinking water plan? Download guide in English Spanish French Arabic or Vietnamese. http://t.co/S0ktilisKq

---

Target: 0 (not real disaster
Text:
Seen on Fahlo:#WCW All Hail the QueenåÊ?? http://t.co/oLpBmy9xw9 #MTVHottest Justin Bieber http://t.co/ON18cqGcoA

---

Target: 0 (not real disaster
Text:
Aftershock was the most terrifying best roller coaster I've ever been on. *DISCLAIMER* I've been on very few.

---

Target: 1 (real disaster)
Text:
Severe thunderstorm warning remains for #Cochrane. @cityofcalgary has enacted municipal emergency plan after today's storm. #abstorm

---



### Split data into training and validation sets

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
# Use train_test_split top split training data into training and validation sets

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(), train_df_shuffled['target'].to_numpy(),
                                                                            test_size = 0.1, random_state = 42)

In [13]:
# Check the lengths

len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [14]:
# Check the first 10 samples

train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## Converting text into numbers

When dealing with a text problem, one of the first things you'll have to do before you can build a midek is to convert your text to numbers.

There are a few ways to do this, namely:
* Tokenization - direct mapping of  token( a toke could be a word or a character) to number
* embedding - create a matrix of feature vectors for each token the size of the feature vector can be defined and this embedding can be learned

### Text vectorization (tokenization)

In [15]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [16]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization parameters

text_vectorizer = TextVectorization(max_tokens = None, # how many words in the vocab (automatically add <OOV>)
                                    standardize = "lower_and_strip_punctuation",
                                    split = "whitespace", 
                                    ngrams = None,
                                    output_mode = "int", #how to map tokens to numbers
                                    output_sequence_length = None, #pads each sequence to the logest sequence
                                    pad_to_max_tokens = True)

In [17]:
# Find the average number of tokens (words) in the training tweets

round(sum(len(i.split()) for i in train_sentences)) / len(train_sentences)

14.901036345059115

In [18]:
# Setup text vectorization variables

max_vocab_length = 10000 #max num of words to have in our vocabulary
max_length = 15 # max length our seuqences will be (how many words in a tweet, amodel would see)

text_vectorizer = TextVectorization(max_tokens = max_vocab_length, output_mode= 'int', output_sequence_length = max_length)

In [19]:
#fit the text vectorier to the training text

text_vectorizer.adapt(train_sentences)

In [20]:
# Create a sample sentence and tokenize it

sample_sentence = "Theres a flood in my street!"

text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [21]:
# Choose a random sentence from the training dataset and tokenize iot

random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
 'If you go on with this nuclear arms race all you are going to do is make the rubble bounce.' ? Winston Churchill

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  47,   12,  112,   11,   14,   19,  105, 2192, 2768,   44,   12,
          22,  104,    5,   68]])>

In [22]:
# Get the unique words in the vocabulary

words_in_vocab = text_vectorizer.get_vocabulary() #get all of the unique words in the training data
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] #get the least common words
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most comon words: {top_5_words}")

Number of words in vocab: 10000
5 most comon words: ['', '[UNK]', 'the', 'a', 'in']


### Creating an Embedding using an Embedding Layer

To make our embedding, we're going to use tensoflows embedding layer

The parameters we actually care about:
* `input_dim` = the size of our vocabulary
* `output_dim` = the size of the output embedding vector, for example, a value of 100 would mean each token gets represented by a vecotr 100 long
* `input_length` = length of the sequences being passed to the embedding layer

In [23]:
from tensorflow.keras import layers

# in keras, wordks best when numbers are divisible by eight

embedding = layers.Embedding(input_dim = max_vocab_length,
                             output_dim = 128,
                             input_length = max_length
                             )
embedding

In [24]:
# Get a random sentence from the training set

random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\n\nEmbedded version:")

# Embed the random sentence (turn into dense vecotrs of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 @Epic_Insanity It got derailed outside Grimrail Depot...

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.01047666, -0.04618541,  0.00207051, ..., -0.03424738,
          0.01741663,  0.01074405],
        [-0.03328212,  0.00590597,  0.02693787, ...,  0.00047261,
         -0.02006059,  0.024491  ],
        [-0.03996246, -0.04608276,  0.01444763, ..., -0.01359117,
          0.01808906,  0.03372772],
        ...,
        [ 0.04286379,  0.00187195,  0.04902841, ...,  0.0480523 ,
         -0.01340004, -0.02682008],
        [ 0.04286379,  0.00187195,  0.04902841, ...,  0.0480523 ,
         -0.01340004, -0.02682008],
        [ 0.04286379,  0.00187195,  0.04902841, ...,  0.0480523 ,
         -0.01340004, -0.02682008]]], dtype=float32)>

In [25]:
# Chekc out a single token's embedding

sample_embed[0][0], sample_embed[0][0].shape, random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.01047666, -0.04618541,  0.00207051, -0.01999754, -0.04905098,
         0.04015628,  0.02418551, -0.03234537,  0.00053991,  0.00679702,
         0.00191369,  0.0440807 ,  0.00900842,  0.0152978 ,  0.03850639,
         0.02259849, -0.01668298,  0.04545177, -0.02773161,  0.02805177,
         0.03645641,  0.02132385,  0.0216934 ,  0.04798469,  0.02672093,
         0.04031323, -0.03660306,  0.04878786,  0.04746426, -0.02749999,
         0.0021134 ,  0.0372741 , -0.00900165, -0.01320041,  0.02958525,
         0.04388133, -0.04145175, -0.01107869,  0.0428392 , -0.03554166,
         0.03062192,  0.03238335,  0.02182421,  0.02241507, -0.00072523,
         0.02374176,  0.00877333, -0.00794033, -0.04222196,  0.01164725,
         0.03418603,  0.00998669,  0.02106657,  0.00259709,  0.0296754 ,
         0.01083484,  0.01395942, -0.01072314, -0.00509969,  0.04971688,
        -0.00755472,  0.01016914,  0.02416697, -0.01494027, -0.00268295,
  

## Modelling a text dataset (running a series of experiments)

Now we're got a wat to turn our text srequences into numbers, it's time to start building a series of modelling experiments, start from a baseline and move on from there 

* Model 0: Naive Bayes (baseline)
* Model 1: Feed-forward neual network (dense model)
* Model 2: GRU model (RNN)
* Model 4: Bidirectional-LSTM
* Model 5: 1d Convolutional Neural Network (CNN)
* Model 6: Tensorflow Hub Pretrained Geature Extractor (using transfer learning for NLP)
* Model 7: Same as model 6 with 10% of training data

How are we going to approach all of these?

Use the standard steps in modelling with tensorflow:

* Create a model
* Build a model
* Compile a model
* Fit the model

### Model 0: Getting a baseline

As with all machine learning modelling experiments it's important to create a baseline model so u got a benchmark for future experiments to build upon

To create our baseline, we'll use SKlearns Multinomial Naive Bayes using the TF-IDF formula to convert our words to numbers

> **Note:** It's common practive to use non-DL algorithms as a baseline because of their speed, then later using DL to improve

In [26]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline

model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()),
                    ("clf", MultinomialNB())
])

# Fit the pipeline to the training data

model_0.fit(train_sentences, train_labels)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [27]:
# Evaluate our baseline model 

baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [28]:
# Make predictions

baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function for our model experiments

We could evaluate model's predictin with different methods every time, but this could be cumbersome and could easily be fixed with a function

Let's create one to compare our model's predictions with the truth labels using the following metrics:

* Accuracy
* Precision
* Recall
* F1-score

In [29]:
# Function to evaluate: accuracty, precision, recall, f1-score

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calulates model accuracy, precision, recall and f1 score of a binary classification model.
  """
  #Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  #Calculate model precision, recall ,and f01 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average = 'weighted')
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}
  return model_results

In [30]:
baseline_results = calculate_results(y_true = val_labels, y_pred = baseline_preds)

baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

### Model 1: A simple dense model

In [31]:
# Create a tensorboard callback (need to create a new one for each model)

from helper_functions import create_tensorboard_callback

# Create a directory to save TensorBoard logs

SAVE_DIR = "model_logs"

In [32]:
from tensorflow.keras import layers

inputs = layers.Input(shape = (1,), dtype = tf.string) # inputs are 1-dimensional
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numberized imputs
x = layers.GlobalAveragePooling1D()(x) # condense the feature vector for each token to one vector
#x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1, activation = "sigmoid")(x) # Create the output layer, want binary output so use sigmoid

model_1 = tf.keras.Model(inputs, outputs, name = "model_1_dense")

In [33]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [34]:
# Compile mode

model_1.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(), metrics = ["accuracy"])

In [35]:
# Fit the mode
model_1_history = model_1.fit(x = train_sentences, y = train_labels, epochs = 15, validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR, experiment_name = "model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20210526-194301
Epoch 1/15
215/215 [==============================] - 7s 17ms/step - loss: 0.6092 - accuracy: 0.6910 - val_loss: 0.5352 - val_accuracy: 0.7507
Epoch 2/15
215/215 [==============================] - 3s 14ms/step - loss: 0.4408 - accuracy: 0.8213 - val_loss: 0.4733 - val_accuracy: 0.7822
Epoch 3/15
215/215 [==============================] - 3s 14ms/step - loss: 0.3468 - accuracy: 0.8597 - val_loss: 0.4605 - val_accuracy: 0.7940
Epoch 4/15
215/215 [==============================] - 3s 14ms/step - loss: 0.2840 - accuracy: 0.8924 - val_loss: 0.4671 - val_accuracy: 0.7927
Epoch 5/15
215/215 [==============================] - 3s 14ms/step - loss: 0.2370 - accuracy: 0.9124 - val_loss: 0.4795 - val_accuracy: 0.7769
Epoch 6/15
215/215 [==============================] - 3s 14ms/step - loss: 0.2005 - accuracy: 0.9264 - val_loss: 0.5025 - val_accuracy: 0.7848
Epoch 7/15
215/215 [==============================] - 3s 13ms/step -

In [36]:
# Check the results

model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.7732 - accuracy: 0.7703


[0.7732366323471069, 0.7703412175178528]

In [37]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

(762, 1)

In [38]:
# look at a single prediction

model_1_pred_probs[0] # anything below 0.5 = disaster, abouve = non-=disaster

array([0.480202], dtype=float32)

In [39]:
# look at the first 10 predictions

model_1_pred_probs[:10]

array([[0.480202  ],
       [0.8405013 ],
       [0.99907374],
       [0.08197001],
       [0.00123418],
       [0.98816645],
       [0.8344143 ],
       [0.99998295],
       [0.99994135],
       [0.73105735]], dtype=float32)

In [40]:
# Convert model prediction probablilities to label format

model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))

model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [41]:
# calculate our model_1 results

model_1_results = calculate_results(y_true = val_labels, y_pred = model_1_preds)

model_1_results

{'accuracy': 77.03412073490814,
 'f1': 0.7684486602580174,
 'precision': 0.7715893693867238,
 'recall': 0.7703412073490814}

In [42]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

In [43]:
import numpy as np

np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

# Visualizing learned embeddings


In [44]:
# get the vocab from the text vecotrization layer

words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [45]:
# Model 1 summary

model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [46]:
# Get the wieght matrix of embedding layer
# (these are the numerical representations of each token in our training data, which have been learned for 5 epocks)

embed_weights = model_1.get_layer("embedding").get_weights()[0]

print(embed_weights.shape) # same size as vocab size and embedding dim (output dim of our embedding layer)

(10000, 128)


In [47]:
embed_weights

array([[ 0.03287848, -0.01077829,  0.0398485 , ...,  0.05907033,
        -0.00235764, -0.03854776],
       [ 0.00970599, -0.04863368,  0.00415329, ..., -0.03332829,
         0.01916864,  0.00813477],
       [-0.01269921, -0.03966425, -0.04843201, ..., -0.01803764,
        -0.02748964, -0.00289053],
       ...,
       [ 0.0262516 ,  0.01350881,  0.02646807, ...,  0.01978141,
         0.046014  , -0.03561642],
       [-0.06411702, -0.06825753, -0.06583641, ...,  0.05338053,
         0.02562852, -0.03883247],
       [-0.13784286, -0.10867371, -0.18086028, ...,  0.18514158,
         0.14909989, -0.11820889]], dtype=float32)

In [48]:
# Create embedding files

import io

out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

#for index, word in enumerate(words_in_vocab):
  #if index == 0:
   # continue  # skip 0, it's padding.
  #vec = embed_weights[index]
  #out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  #out_m.write(word + "\n")
#out_v.close()
#out_m.close()

In [49]:
# Download files from Colab to upload to projector

#try:
  #from google.colab import files
  #files.download('vectors.tsv')
  #files.download('metadata.tsv')
#except Exception:
  #pass

## Recurrent Neural Networks (RNN's)

RNN's are usefull for sequence data.

The premise of a recurrent neural network is to use the representation of a previous input to aid the respresentation of a later input.

### Model 2: LSTM

LSTM = ling short term memory (one of the most popular LSTM cells)

Our structure of an RNN typically looks like this:

'''
Input (text) -> Tokenize -> EMbedding -> Layers (RNN's/dense) -> Output (label probability)
'''

In [50]:
# Create an LSTM model

from tensorflow.keras import layers

inputs = layers.Input(shape = (1,), dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
#x = layers.LSTM(64, return_sequences = True)(x)
print(x.shape)
x = layers.LSTM(64)(x) # when you're stacking RNN cells together, you need to return_sequences = true
print(x.shape)
#x = layers.Dense(64, activation = 'relu')(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)

model_2 = tf.keras.Model(inputs, outputs, name = "model_2_LSTM")

(None, 15, 128)
(None, 15, 128)
(None, 64)


In [51]:
# Compile the model

model_2.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(), metrics = ["accuracy"])

In [52]:
model_2_history = model_2.fit(train_sentences, train_labels, epochs = 5, 
                              validation_data = (val_sentences, val_labels), callbacks = [create_tensorboard_callback(SAVE_DIR, "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20210526-194425
Epoch 1/5
215/215 [==============================] - 10s 22ms/step - loss: 0.1188 - accuracy: 0.9657 - val_loss: 0.8207 - val_accuracy: 0.7664
Epoch 2/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0673 - accuracy: 0.9740 - val_loss: 0.9200 - val_accuracy: 0.7717
Epoch 3/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0624 - accuracy: 0.9774 - val_loss: 0.9336 - val_accuracy: 0.7664
Epoch 4/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0581 - accuracy: 0.9759 - val_loss: 1.0934 - val_accuracy: 0.7664
Epoch 5/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0494 - accuracy: 0.9775 - val_loss: 1.0036 - val_accuracy: 0.7598


In [53]:
# Make predictions with LSTM model

model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]


array([[1.07230075e-01],
       [7.59281218e-01],
       [9.96703804e-01],
       [2.07418561e-01],
       [2.94627709e-04],
       [9.95440841e-01],
       [2.77492911e-01],
       [9.99923110e-01],
       [9.99914646e-01],
       [9.86454308e-01]], dtype=float32)

In [54]:
# Convert model 2 pred probs to labels

model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 0., 1., 1., 1.], dtype=float32)>

In [55]:
model_2_results = calculate_results(y_true = val_labels, y_pred = model_2_preds)
model_2_results

{'accuracy': 75.98425196850394,
 'f1': 0.758116023760764,
 'precision': 0.7604578907479682,
 'recall': 0.7598425196850394}

### Model 3: GRU

Another poppular and effective RNN component is the GRU or gated recurrent unit.

The GRU cell has similar features to an LSTM cell but has less parameters.

In [56]:
# Build an RNN using the GRU cell
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
#x = layers.LSTM(64, return_sequences = True)(x)
#print(x.shape)
#x = layers.GRU(64)(x) # make sure return seuqences DOES NO equal true OR add a global average pooling layer instead
#x = layers.Dense(64, activation = "relu")(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)


model_3 = tf.keras.Model(inputs, outputs)

In [57]:
 model_3.summary(
     
 )

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
gru (GRU)                    (None, 64)                37248     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_________________________________________________________________


In [58]:
# COmpile the model

model_3.compile(loss = "binary_crossentropy", optimizer= tf.keras.optimizers.Adam(), metrics = ["accuracy"])


In [59]:
# Fit

model_3_history = model_3.fit(train_sentences,
                              train_labels, epochs = 10,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR, "model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20210526-194508
Epoch 1/10
215/215 [==============================] - 6s 20ms/step - loss: 0.1249 - accuracy: 0.9520 - val_loss: 0.8516 - val_accuracy: 0.7651
Epoch 2/10
215/215 [==============================] - 3s 16ms/step - loss: 0.0581 - accuracy: 0.9769 - val_loss: 0.9056 - val_accuracy: 0.7598
Epoch 3/10
215/215 [==============================] - 3s 15ms/step - loss: 0.0521 - accuracy: 0.9783 - val_loss: 0.8725 - val_accuracy: 0.7638
Epoch 4/10
215/215 [==============================] - 3s 16ms/step - loss: 0.0483 - accuracy: 0.9793 - val_loss: 1.0823 - val_accuracy: 0.7598
Epoch 5/10
215/215 [==============================] - 3s 16ms/step - loss: 0.0416 - accuracy: 0.9804 - val_loss: 1.1240 - val_accuracy: 0.7598
Epoch 6/10
215/215 [==============================] - 3s 16ms/step - loss: 0.0396 - accuracy: 0.9810 - val_loss: 1.3030 - val_accuracy: 0.7703
Epoch 7/10
215/215 [==============================] - 3s 15ms/step - l

In [60]:
# Make some predictions with GRu model

model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

array([[2.3839268e-05],
       [6.7373121e-01],
       [9.9992394e-01],
       [2.0152812e-01],
       [1.2908875e-05],
       [9.9995506e-01],
       [9.9915314e-01],
       [9.9998236e-01],
       [9.9997246e-01],
       [9.9984705e-01]], dtype=float32)

In [61]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [62]:
# Calculate model 3 results

model_3_results = calculate_results(y_true = val_labels, y_pred = model_3_preds)
model_3_results

{'accuracy': 76.77165354330708,
 'f1': 0.7646846187166754,
 'precision': 0.7712255031085189,
 'recall': 0.7677165354330708}

### Model 4: Bidirectional RNN

ormal RNN's go from left to right (just like you'd read an English sentence) however, a bidiractional RNN goes from right to left as well as left to right.

In [63]:
# Build a dirirectional RNN in Tensorflow

from tensorflow.keras import layers

inputs = layers.Input(shape = (1,), dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
#x = layers.Bidirectional(layers.LSTM(64, return_sequences = True))(x)
#print(x.shape)
x = layers.Bidirectional(layers.GRU(64))(x)
print(x.shape)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_4 = tf.keras.Model(inputs, outputs, name = "model_4_bidirectional")

(None, 128)


In [64]:
# Get a summary

model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               74496     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 1,354,625
Trainable params: 1,354,625
Non-trainable params: 0
_________________________________________________________________


In [65]:
# Compile model

model_4.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(), metrics = ['accuracy'])

In [66]:
model_4_history = model_4.fit(train_sentences, train_labels, epochs = 5, validation_data = (val_sentences, val_labels), callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                                                                                                                                 "model_4_bidirectional")])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20210526-194546
Epoch 1/5
215/215 [==============================] - 7s 23ms/step - loss: 0.0994 - accuracy: 0.9679 - val_loss: 1.0387 - val_accuracy: 0.7585
Epoch 2/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0470 - accuracy: 0.9801 - val_loss: 1.2397 - val_accuracy: 0.7572
Epoch 3/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0408 - accuracy: 0.9800 - val_loss: 1.2391 - val_accuracy: 0.7598
Epoch 4/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0360 - accuracy: 0.9813 - val_loss: 1.2790 - val_accuracy: 0.7625
Epoch 5/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0334 - accuracy: 0.9826 - val_loss: 1.3402 - val_accuracy: 0.7638


In [67]:
# Make predictions with out bidiractional ,odel
 
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

array([[8.7149558e-04],
       [7.9716021e-01],
       [9.9968314e-01],
       [2.6114169e-01],
       [2.5018046e-05],
       [9.9976212e-01],
       [9.7900140e-01],
       [9.9997067e-01],
       [9.9993122e-01],
       [9.9914038e-01]], dtype=float32)

In [68]:
# Convert

model_4_pred = tf.squeeze(tf.round(model_4_pred_probs))
model_4_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [69]:
# Calculate the results of our bidiractional model

model_4_results = calculate_results(y_true = val_labels, y_pred = model_4_pred )
model_4_results

{'accuracy': 76.37795275590551,
 'f1': 0.7621412379223811,
 'precision': 0.764383846466808,
 'recall': 0.7637795275590551}

### Model 5: Conv1D 

We've used CNNs for images but images are typically 2D (hegith x width)...however, our text data is 1d.

The typical structure of a Conv1D model for sequen ces (in our case, text):

```
Inputs(test -> Tokenization -> Embedding -> Cpmv1D -> Pool -> Outputs (class probs)
```

In [70]:
from tensorflow.keras import layers

input = layers.Input(shape = (1,), dtype = "string")
x = text_vectorizer(input)
x = embedding(x)
x = layers.Conv1D(32, 6, activation = 'relu')(x) #kernal size is reffered to as n-gram side (strides rn is 1, but can change to hop) #padding = 'valid' (some words mised) output is smaller 'same' = smae size
x = layers.GlobalMaxPooling1D()(x)
output = layers.Dense(1, activation = "sigmoid")(x)

model_5 = tf.keras.Model(input, output, name = "model_5_conv")


In [71]:
model_5.summary()

Model: "model_5_conv"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
conv1d (Conv1D)              (None, 10, 32)            24608     
_________________________________________________________________
global_max_pooling1d (Global (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 1,304,641
Trainable params: 1,304,641
Non-trainable params: 0
____________________________________________

In [72]:
# Compiling the model

model_5.compile(loss = 'binary_crossentropy', optimizer = tf.keras.optimizers.Adam(), metrics = ['accuracy'])



In [73]:
# Fitting the model

model_5_history = model_5.fit(train_sentences, train_labels, epochs = 5, 
                              validation_data = (val_sentences, val_labels), callbacks = [create_tensorboard_callback(SAVE_DIR, "model_5_conv")])

Saving TensorBoard log files to: model_logs/model_5_conv/20210526-194610
Epoch 1/5
215/215 [==============================] - 30s 20ms/step - loss: 0.1034 - accuracy: 0.9672 - val_loss: 1.0296 - val_accuracy: 0.7598
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0601 - accuracy: 0.9765 - val_loss: 1.1526 - val_accuracy: 0.7520
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0508 - accuracy: 0.9788 - val_loss: 1.2415 - val_accuracy: 0.7480
Epoch 4/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0485 - accuracy: 0.9781 - val_loss: 1.3180 - val_accuracy: 0.7533
Epoch 5/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0455 - accuracy: 0.9784 - val_loss: 1.3654 - val_accuracy: 0.7480


In [74]:
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters = 64, kernel_size = 5, activation = "relu")(x)
x = layers.GlobalMaxPool1D()(x)
#x = layers.Dense(64, activation = "relu")(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)
model_6 = tf.keras.Model(inputs, outputs, name = "model_6_Conv1D")

model_6.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(), metrics = ['accuracy'])

model_6.summary()

Model: "model_6_Conv1D"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 11, 64)            41024     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 1,321,089
Trainable params: 1,321,089
Non-trainable params: 0
__________________________________________

In [75]:
# Fit the model

model_6_history = model_6.fit(train_sentences, train_labels, epochs = 5,
                              validation_data = (val_sentences, val_labels), callbacks = [create_tensorboard_callback(SAVE_DIR, "Conv1D")])

Saving TensorBoard log files to: model_logs/Conv1D/20210526-194733
Epoch 1/5
215/215 [==============================] - 5s 18ms/step - loss: 0.0772 - accuracy: 0.9750 - val_loss: 1.1440 - val_accuracy: 0.7559
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0485 - accuracy: 0.9783 - val_loss: 1.2864 - val_accuracy: 0.7507
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0443 - accuracy: 0.9804 - val_loss: 1.3164 - val_accuracy: 0.7493
Epoch 4/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0435 - accuracy: 0.9807 - val_loss: 1.3506 - val_accuracy: 0.7415
Epoch 5/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0423 - accuracy: 0.9807 - val_loss: 1.3858 - val_accuracy: 0.7493


In [76]:
# Make some prediction with our Conv1D model

model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred = tf.squeeze(tf.round(model_6_pred_probs))
model_6_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [77]:
model_6_results = calculate_results(y_true = val_labels, y_pred = model_6_pred)
model_6_results

{'accuracy': 74.93438320209974,
 'f1': 0.7474119306358412,
 'precision': 0.749916614970861,
 'recall': 0.7493438320209974}

## Model6: Tensorflow Hub Pretrained Sentence Encoder

Using transfer learning for NLP

In [78]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_samples = embed([sample_sentence, "When you call the use on a sentence, it utrns it into numbers"])

print(embed_samples[0][:50])

tf.Tensor(
[-0.01157027  0.0248591   0.02878048 -0.012715    0.03971538  0.0882776
  0.02680985  0.05589838 -0.01068729 -0.00597292  0.00639323 -0.0181952
  0.00030814  0.09105888  0.05874645 -0.03180628  0.01512474 -0.05162929
  0.00991367 -0.06865346 -0.04209305  0.0267898   0.03011008  0.00321069
 -0.00337971 -0.04787356  0.02266719 -0.00985925 -0.04063613 -0.01292093
 -0.04666384  0.056303   -0.03949255  0.00517688  0.02495828 -0.07014441
  0.02871508  0.04947684 -0.00633978 -0.08960193  0.02807117 -0.00808362
 -0.01360601  0.0599865  -0.10361787 -0.05195374  0.00232955 -0.0233253
 -0.03758105  0.03327729], shape=(50,), dtype=float32)


In [79]:
# transforms THE WHOLE sentence into a 512 dimensional vector

In [80]:
# Create Keras Layer using the USE pretrained layer from tensorflow hub

sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape = [],
                                        dtype = tf.string,
                                        trainable = False,
                                        name = "USE")

In [81]:
# Create model using the sequenctial API

model_7 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation = "relu"),
    layers.Dense(1, activation = 'sigmoid', name = 'output_layer')
], name = "model_7_USE")

# Compile

model_7.compile(loss = 'binary_crossentropy', optimizer = 
                tf.keras.optimizers.Adam(), metrics = ['accuracy'])

model_7.summary()

Model: "model_7_USE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
USE (KerasLayer)             (None, 512)               256797824 
_________________________________________________________________
dense_6 (Dense)              (None, 64)                32832     
_________________________________________________________________
output_layer (Dense)         (None, 1)                 65        
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [82]:
# Train a classifier on top of USE pretrained embeddings

model_7_history = model_7.fit(train_sentences, train_labels, epochs = 15,
                              validation_data = (val_sentences, val_labels), callbacks = [create_tensorboard_callback(SAVE_DIR, "model_7_use")])

Saving TensorBoard log files to: model_logs/model_7_use/20210526-194815
Epoch 1/15
215/215 [==============================] - 7s 23ms/step - loss: 0.5044 - accuracy: 0.7853 - val_loss: 0.4486 - val_accuracy: 0.7979
Epoch 2/15
215/215 [==============================] - 2s 10ms/step - loss: 0.4157 - accuracy: 0.8158 - val_loss: 0.4401 - val_accuracy: 0.8071
Epoch 3/15
215/215 [==============================] - 2s 11ms/step - loss: 0.4025 - accuracy: 0.8206 - val_loss: 0.4345 - val_accuracy: 0.8110
Epoch 4/15
215/215 [==============================] - 2s 11ms/step - loss: 0.3946 - accuracy: 0.8256 - val_loss: 0.4326 - val_accuracy: 0.8123
Epoch 5/15
215/215 [==============================] - 2s 10ms/step - loss: 0.3871 - accuracy: 0.8311 - val_loss: 0.4338 - val_accuracy: 0.8163
Epoch 6/15
215/215 [==============================] - 2s 10ms/step - loss: 0.3811 - accuracy: 0.8318 - val_loss: 0.4332 - val_accuracy: 0.8136
Epoch 7/15
215/215 [==============================] - 2s 10ms/step - l

In [83]:
model_7_pred_probs = model_7.predict(val_sentences)
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
model_7_results = calculate_results(val_labels, model_7_preds)


## Model8 :TF Hub Pretrained USE but with 10% of training data

Transfer learning really helps when you don't have a alarge dataset.

To see how our model performs on a smaller dataset, let's replicate 'model_& except we'll train it on 10% of the data.

In [84]:
# Create subset of 10% of the training data

# NOte

#train_10_percent = train_df_shuffled[["text", "target"]].sample(frac = 0.1, random_state = 42)

#train_sentences_10_percent = train_10_percent["text"].to_list()
#train_labels_10_percent = train_10_percent['target'].to_list()

 **Note** Be *very* carefule when creating training/va;/test splits that you don't leak data across the datasets, otherwise your model evaluation metrics will be wrong. If something looks to good to be true (a model trainied on 10 percent of data doutperforming a mdel trained on100% of the data) trust your gut and o back thorugh to find where the error may lie.

In [85]:
train_10_percent_split = int(0.1* len(train_sentences))
train_10_percent = train_sentences[:train_10_percent_split]
train_labels_10_percent =  train_labels[:train_10_percent_split]

In [86]:
pd.Series(np.array(train_labels_10_percent)).value_counts()

0    406
1    279
dtype: int64

In [87]:
# Check the number of target in our subset of data (ensure ratios are relatively close)

train_df_shuffled["target"].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [88]:
train_df_shuffled['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

To recreeate amodel the same as a previous model you've created, you can use the 'tf.keras.models.clone_model ' method

In [89]:
# Let's build a nodel the same as model_6

model_8 = tf.keras.models.clone_model(model_7)

model_8.compile(loss = 'binary_crossentropy', optimizer= tf.keras.optimizers.Adam(), metrics = ['accuracy'])

model_8.summary()

Model: "model_7_USE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
USE (KerasLayer)             (None, 512)               256797824 
_________________________________________________________________
dense_6 (Dense)              (None, 64)                32832     
_________________________________________________________________
output_layer (Dense)         (None, 1)                 65        
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [90]:
# Fit the model on 10% training data

model_8_history = model_8.fit(train_10_percent, train_labels_10_percent, epochs = 5,
                              validation_data = (val_sentences, val_labels), callbacks = [create_tensorboard_callback(SAVE_DIR, 'model_8_correct_split')])

Saving TensorBoard log files to: model_logs/model_8_correct_split/20210526-194857
Epoch 1/5
22/22 [==============================] - 4s 114ms/step - loss: 0.6758 - accuracy: 0.6526 - val_loss: 0.6511 - val_accuracy: 0.7310
Epoch 2/5
22/22 [==============================] - 0s 19ms/step - loss: 0.6038 - accuracy: 0.8015 - val_loss: 0.5942 - val_accuracy: 0.7598
Epoch 3/5
22/22 [==============================] - 1s 24ms/step - loss: 0.5279 - accuracy: 0.8117 - val_loss: 0.5389 - val_accuracy: 0.7717
Epoch 4/5
22/22 [==============================] - 0s 20ms/step - loss: 0.4656 - accuracy: 0.8292 - val_loss: 0.5042 - val_accuracy: 0.7717
Epoch 5/5
22/22 [==============================] - 0s 19ms/step - loss: 0.4242 - accuracy: 0.8307 - val_loss: 0.4910 - val_accuracy: 0.7677


In [91]:
model_8_pred_probs = model_8.predict(val_sentences)

model_8_preds = tf.squeeze(tf.round(model_8_pred_probs))

model_8_results = calculate_results(val_labels, model_8_preds)

#print(results)

## Comparing the performance of each of our models

In [92]:
# Combine model results into a Datafrmae 

all_model_results = pd.DataFrame({"0_baseline": baseline_results, "1_simple_dense": model_1_results,
                                  "2_lstm": model_2_results, "3_gru": model_3_results,
                                  "4_bidirectional": model_4_results, "5_conv1d": model_6_results, 
                                  "6_tf_hub_use_encoder": model_7_results, 
                                  "7_tf_hub_use_encoder_10_percent":model_8_results})
all_model_results

,0_baseline,1_simple_dense,2_lstm,3_gru,4_bidirectional,5_conv1d,6_tf_hub_use_encoder,7_tf_hub_use_encoder_10_percent
accuracy,79.265092,77.034121,75.984252,76.771654,76.377953,74.934383,81.889764,76.771654
precision,0.811139,0.771589,0.760458,0.771226,0.764384,0.749917,0.823311,0.768688
recall,0.792651,0.770341,0.759843,0.767717,0.763780,0.749344,0.818898,0.767717
f1,0.786219,0.768449,0.758116,0.764685,0.762141,0.747412,0.816825,0.765926


## Uploading model's training logs to TensorBoard.dev

In [93]:
!tensorboard dev upload --logdir ./model_logs/ \
 --name "NLP Modelling Experiments" \
 --description "comparing model architectures" \
 --one_shot # exit the uploader once finished

2021-05-26 19:49:06.940058: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0

***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

./model_logs/

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) yes

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=373649185512-8v619h5kft38l4456nm2dj4ubeqsrvh6.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3

Now I've ran the cell above, my modelling experiments are visable on TensorBoard.dev:

for larger sclae experiment and more tracking options, check out weights and biases

In [94]:
# If you need to delete an experiment from TensorBoard, you can run the following:

# See the previous Tensorboard Dev experiments you've run

!tensorboard dev list

2021-05-26 19:51:40.574356: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0

***** TensorBoard Uploader *****

This will list all experiments that you've uploaded to
https://tensorboard.dev. TensorBoard.dev experiments are visible
to everyone. Do not upload sensitive data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) Traceback (most recent call last):



In [95]:
# !tensorboard dev delete --exmerimetn_id (take id)

## Saving and loadsing a trained model

There are two main formats to save a model to in Tensorflow:
1. The HDF5 format
2. The `SavedModel` format

In [96]:
# Save TF Hub Sentence Encoder model to HDF5 format

model_7.save("model_7.h5")

In [97]:
loaded_model_6 = tf.keras.models.load_model('model_6.h5',
                                            custom_objects = {"KerasLayers": hub.KerasLayer})

OSError: ignored

# Now let's save to the `SavedModel` format

In [ ]:
model_6.save("model_6_SavedModel_format")

In [ ]:
loaded_model_6_SavedModel_format = tf.keras.models.load_model("")

## Finding the most wrong examples

In [98]:
# Download a dataframe from google storage

!wget https://storage.googleapis.com/ztm_tf_course/08_model_6_USE_feature_extractor.zip
!unzip 08_model_6_USE_feature_extractor.zip

--2021-05-26 20:02:06--  https://storage.googleapis.com/ztm_tf_course/08_model_6_USE_feature_extractor.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 74.125.199.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 960779165 (916M) [application/zip]
Saving to: ‘08_model_6_USE_feature_extractor.zip’

08_model_6_USE_feat 100%[===================>] 916.27M   244MB/s    in 3.8s    

2021-05-26 20:02:10 (239 MB/s) - ‘08_model_6_USE_feature_extractor.zip’ saved [960779165/960779165]

Archive:  08_model_6_USE_feature_extractor.zip
   creating: 08_model_6_USE_feature_extractor/
   creating: 08_model_6_USE_feature_extractor/assets/
   creating: 08_model_6_USE_feature_extractor/variables/
  inflating: 08_model_6_USE_feature_extractor/variables/variables.data-00000-of-00001  
  inflating: 08_model_6_USE_feature_extractor/variables/variables.inde

In [ ]:
# Create a DataFrame with validation sentences and best performing model predictions

In [99]:
# Import previously trained model from google storage

model_6_pretrained = tf.keras.models.load_model("08_model_6_USE_feature_extractor")
model_6_pretrained.evaluate(val_sentences,val_labels)

24/24 [==============================] - 1s 9ms/step - loss: 0.4272 - accuracy: 0.8163


[0.42723122239112854, 0.8162729740142822]

In [102]:
model_6_pretrained_pred_probs = model_6_pretrained.predict(val_sentences)
model_6_pretrained_pred = tf.squeeze(tf.round(model_6_pretrained_pred_probs))
model_6_pretrained_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [104]:
val_df = pd.DataFrame({"text": val_sentences,
                       "target": val_labels,
                       "pred": model_6_pretrained_pred,
                       "pred_prob": tf.squeeze(model_6_pretrained_pred_probs)})
val_df.head()

,text,target,pred,pred_prob
0,DFR EP016 Monthly Meltdown - On Dnbheaven 2015...,0,0.0,0.159757
1,FedEx no longer to transport bioterror germs i...,0,1.0,0.747162
2,Gunmen kill four in El Salvador bus attack: Su...,1,1.0,0.988749
3,@camilacabello97 Internally and externally scr...,1,0.0,0.196229
4,Radiation emergency #preparedness starts with ...,1,1.0,0.707808


In [105]:
# find the wrong predictions and sort by prediction probabilities

most_wrong = val_df[val_df['target'] != val_df['pred']].sort_values("pred_prob", ascending = False)
most_wrong.head() # false positives

,text,target,pred,pred_prob
31,? High Skies - Burning Buildings ? http://t.co...,0,1.0,0.910196
759,FedEx will no longer transport bioterror patho...,0,1.0,0.876982
628,@noah_anyname That's where the concentration c...,0,1.0,0.852300
209,Ashes 2015: AustraliaÛªs collapse at Trent Br...,0,1.0,0.835454
251,@AshGhebranious civil rights continued in the ...,0,1.0,0.827213


In [106]:
most_wrong.tail() # false negatives

,text,target,pred,pred_prob
411,@SoonerMagic_ I mean I'm a fan but I don't nee...,1,0.0,0.043918
233,I get to smoke my shit in peace,1,0.0,0.042087
38,Why are you deluged with low self-image? Take ...,1,0.0,0.038998
244,Reddit Will Now QuarantineÛ_ http://t.co/pkUA...,1,0.0,0.038949
23,Ron &amp; Fez - Dave's High School Crush https...,1,0.0,0.037186
